# Get open, high, low and close data from Yahoo Finance using Yahoo Query API
## We will be getting atleast 5 years historical prices for S&P 500 stocks

### Step 1: Import the required libraries
We will be using Yahoo Query API to pull data from Yahoo Finance. To install this library use pip install yahooquery

In [1]:
import yahooquery as yq
import pandas as pd
import csv
import logging #library to create log files
import datetime as dt
from datetime import datetime # To get the current date and time
from datetime import date, timedelta
from pytz import timezone # Get timezone
import csv
import time
import os # To check if the file exists

### Step 2: Load tickers for S&P 500 stocks from csv file 
Note: We have created seperate python script (S&P500Tickers.ipynb) to get S&P 500 stocks and store that in file name is 'S&P500Tickers.csv'

In [2]:
ticker_data = pd.read_csv("S&P500Tickers.csv") # Open csv file and read data in pandas dataframe
tickers_list = ticker_data['Symbol'].tolist() #Get all the tickers in a list

### Step 3: Create a function that pull data for yfinance api for each ticker
OHLC - Open high low and close of a stock on a given date
Script needs start and end date to pull data between the range else data for max period will be pulled
Start date and end date has to be in format YYYY-MM-DD
Add daily percent return [(close - Open) / Open]

In [3]:
def get_OHLC_data(tickers, start_date = None, end_date = None):
    '''
    This function gets daily OHLC data from Yahoo Finance API for the provided number of days. 
    Daily returns are caluclated and stored in Returns column
    In case days is not provided then data sis pulled for maximum number of days
    
    Input Parameters: 
    tickers: List of tickers for which data needs to be extracted
    days: Number of days for which data needs to be pulled
    
    Returns: Dataframe of the extracted data
    '''
    logging.info("Getting OHLC data for days provided")
    i = 0
    j = 50
    while (j<=len(tickers)):
        # Initiate object to get the data from API
        try:
            yq_ticker_obj = yq.Ticker(tickers[i:j], asynchronous = True) # Setting the asynchronous flag as we are pulling data for multiple tickers
            if(start_date != None and end_date != None):
                OHLC_data = yq_ticker_obj.history(start = start_date, end = end_date, interval = "1d", adj_ohlc = True)
            else:
                OHLC_data = yq_ticker_obj.history(period="max", interval = "1d", adj_ohlc = True)

            # Delete split and dividend columns
            OHLC_data = OHLC_data.drop(['dividends','splits'], axis=1, errors='ignore')
            final_OHLC_df = final_OHLC_df.append(OHLC_data)

        except Exception as e:
            pass
        
        i = j
        j += 50
        if(j>len(tickers)):
            extra = j-len(tickers)
            j = j-extra
        time.sleep(.2)
    return final_OHLC_df
    # Add daily percent change on the price whic is [(close price - open price)/open price]
    # OHLC_data['return'] = ((OHLC_data['close'] - OHLC_data['open'])/OHLC_data['open'])*100 
    

### Step 4: Get the last date and dataframe from the existing OHLC csv file
This function will open existing csv file and put the data in existing dataframe and will also return last date

In [4]:
def get_last_date_data(file):
    '''
    This function open existing OHLC file, put that in pandas dataframe and extract the last date from the file
    Input Parameters: Name of the existing csv file that needs to be read in dataframe
    Returns: Last date and pandas dataframe containing data
    '''
    logging.info("Opening existing data file and extracting last date from it")
    
    #old_data_df = pd.read_csv(file, index_col = ["symbol", "date"]) # Read the csv file in a dataframe
    old_data_df = pd.read_csv(file) # Read the csv file in a dataframe
    #display(old_data_df.head())
    
    # Convert Data column to datetime
    old_data_df['date']= pd.to_datetime(old_data_df['date'])
    previous_date = max(old_data_df['date']) # Get the maximum date which is the last date for which data is present
    logging.info(f'Last date in the file is {previous_date}')
    # Converting datetime to only date
    old_data_df['date'] = old_data_df['date'].dt.date
    old_data_df.set_index(["symbol", "date"], inplace=True)
    #display(old_data_df.head())
    return previous_date, old_data_df

### The main function to run all the sub functions

In [24]:
i = 0
j = 50
while (j<len(tickers_list)):
    x = tickers_list[i:j]
    i = j
    j += 50
    if(j>len(tickers_list)):
        extra=j-len(tickers_list)
        j = j-extra
    


len(tickers_list)

505

In [5]:
if __name__ == "__main__":
    
    #tickers_list = ['AAPL', 'MSFT', 'fb', 'AIG', 'AMZN', 'CAT', 'GOOGL', 'A', 'AAL', 'ABC', ]
    #tickers_list = tickers_list[0:50]   
    # Get the next date in YYYY-MM-DD format. As API gives current business date data when next day is entered
    today = pd.Timestamp.now().normalize()
    next_day =  today + timedelta(days=1)
   
    # Create name of the log file
    log_file_name = 'OHLC_logfile_' + str(today).split()[0] +'.log'
    
    # Initialize a log file at the Info level. This is just to ensure smooth debugging in case anything fails
    # %(asctime)s adds the time of creation of the LogRecord
    logging.basicConfig(filename=log_file_name, filemode="w", format='%(asctime)s - %(message)s', level=logging.INFO)
    
    logging.info("In the main function")
    logging.info(f'Processing for date {today}')
    
   # Name of the output file
    OHLC_data_file = "OHLC_data.csv"
    
    logging.info("Check if file already exists")
    
    # Check if the OHLC csv file existing the root folder from where this script is ran
    if os.path.isfile(OHLC_data_file):
        logging.info("OHLC Data file exists. Getting data in dataframe and last date")
        # Get the last date and dataframe
        previous_day, old_data_df = get_last_date_data(OHLC_data_file)
        
        # Get difference between last date and todays date in days. This will be passed to OHLC function
        #date_diff = (today-previous_date).days     
        #no_days = str()+'d' # Days to be passed in OHLC function is in format "1d"
        latest_data = get_OHLC_data(tickers_list, previous_day, next_day)
        # Merge new data with old data and avoid duplicates
        final_df = old_data_df.append(latest_data)
        # Convert date in the dame format and drop duplicates
        final_df.reset_index(inplace=True)
        final_df = final_df.drop_duplicates(subset=['symbol', 'date'], keep='first')
        final_df.set_index(["symbol", "date"], inplace=True)
        #final_df = pd.concat([old_data_df, latest_data])
    else:
        logging.info("OHLC file does not exists. Getting maximum possible data")
        latest_data = get_OHLC_data(tickers_list) # Get the max data
        final_df = latest_data
    
    # Create name of the output file
    logging.info(f'Writing data in the file name {OHLC_data_file}')
    print(final_df)
    final_df.to_csv(OHLC_data_file, mode='w', index=True) #index is False as we don't want to write index in csv file
    

KeyboardInterrupt: 

In [17]:
#final_df['AAPL']
type(final_df)
display(final_df)

volume        low        high       open      close
symbol date                                                                
AAPL   1980-12-12  469033600.0   0.101261    0.101701   0.101261   0.101261
       1980-12-15  175884800.0   0.095978    0.096418   0.096418   0.095978
       1980-12-16  105728000.0   0.088934    0.089374   0.089374   0.088934
       1980-12-17   86441600.0   0.091135    0.091575   0.091135   0.091135
       1980-12-18   73449600.0   0.093777    0.094217   0.093777   0.093777
...                        ...        ...         ...        ...        ...
ABC    2020-10-16    1200600.0  98.900002  100.610001  99.599998  99.519997
       2020-10-19     527400.0  97.180000  100.720001  99.529999  97.800003
       2020-10-20     726100.0  97.400002   99.000000  98.169998  97.989998
       2020-10-21     829500.0  97.730003  100.220001  97.750000  97.730003
       2020-10-22     619400.0  97.220001   98.919998  97.900002  98.720001

[73234 rows x 5 columns]